* 한국공항공사, 인천국제공항공사,「항공통계」 항공사별 통계
* https://kosis.kr/statHtml/statHtml.do?orgId=381&tblId=DT_920005_B008&conn_path=I2

In [ ]:
!pip install koreanize-matplotlib

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib

In [ ]:
from glob import glob

file_names = glob("*company.csv")
file_names = sorted(file_names)
file_names

In [ ]:
df_comp = pd.read_csv(file_names[0], encoding="cp949", dtype={"시점": "object"})
df_comp

In [ ]:
df_comp.info()

In [ ]:
# 항공사별로 연간 평균 여객 수 계산
average_passengers_by_airline = df_comp.groupby("항공사별(1)")["여객 (명)"].mean()

# 그래프로 표시
plt.figure(figsize=(12, 3))
average_passengers_by_airline.plot(kind="bar", color="skyblue")
plt.title("항공사별 연간 평균 여객 수")
plt.xlabel("항공사")
plt.ylabel("평균 여객 수")
plt.xticks(rotation=45)
plt.show()


## 연간 운항 및 여객 증가율 계산:
* 연간 운항 편수 및 여객 수의 증가율을 계산하여 항공사별로 비교합니다.

In [ ]:
# 연간 운항 편수 증가율 계산
df_comp["운항 (편)_증가율"] = df_comp.groupby("항공사별(1)")["운항 (편)"].pct_change() * 100

# 연간 여객 수 증가율 계산
df_comp["여객 (명)_증가율"] = df_comp.groupby("항공사별(1)")["여객 (명)"].pct_change() * 100
df_comp

## 도착/출발별로 연간 운항 및 여객 분석:
* 도착과 출발별로 연간 운항 편수 및 여객 수를 분석합니다

In [ ]:
# 도착별 연간 운항 편수 및 여객 수 분석
arrival_stats = df_comp[df_comp["도착출발별(1)"] == "도착"].groupby("시점")[["운항 (편)", "여객 (명)"]].sum()

# 출발별 연간 운항 편수 및 여객 수 분석
departure_stats = df_comp[df_comp["도착출발별(1)"] == "출발"].groupby("시점")[["운항 (편)", "여객 (명)"]].sum()

In [ ]:
arrival_stats

In [ ]:
departure_stats.plot(secondary_y="여객 (명)")

## 시간에 따른 화물 운송량 시각화:
* 시간에 따른 화물 운송량의 추이를 시각화합니다.

In [ ]:
# 시간에 따른 화물 운송량 시각화
df_comp["시점"] = pd.to_datetime(df_comp["시점"])  # 날짜 형식으로 변환

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df_comp[df_comp["항공사별(1)"] == "아시아나항공"]["시점"], df_comp[df_comp["항공사별(1)"] == "아시아나항공"]["화물 (톤)"], label="아시아나항공")
plt.plot(df_comp[df_comp["항공사별(1)"] == "대한항공"]["시점"], df_comp[df_comp["항공사별(1)"] == "대한항공"]["화물 (톤)"], label="대한항공")
plt.xlabel("시간")
plt.ylabel("화물 운송량 (톤)")
plt.legend()
plt.title("시간에 따른 화물 운송량 추이")
plt.show()

* 데이터를 다시 로드하여 df_comp 초기화

In [ ]:
df_comp = pd.read_csv(file_names[0], encoding="cp949", dtype={"시점": "object"})
df_comp

In [ ]:
df_comp.columns = ['시점', '항공사', '도착출발', '운항_편', '여객_명', '화물_톤']
df_comp

df_comp["연도"] = df_comp["시점"].astype(str).str[:4]:

df_comp DataFrame에 "연도"라는 새로운 열을 추가합니다.
"시점" 열에 있는 데이터를 문자열로 변환한 후, str[:4]를 사용하여 처음 4자리를 추출합니다.
이렇게 추출한 값은 연도를 나타냅니다. 예를 들어, "2005.01"에서 "2005"를 추출합니다.
df_comp["월"] = df_comp["시점"].map(lambda x: x.split(".")[1][:2]):

df_comp DataFrame에 "월"이라는 새로운 열을 추가합니다.
"시점" 열에 있는 데이터를 map 함수와 람다(lambda) 함수를 사용하여 처리합니다.
x.split(".")를 사용하여 "시점" 값을 점 (".")을 기준으로 분할하고, 분할된 결과 중 [1]을 선택하여 월과 연도를 구분합니다.
[1][:2]를 사용하여 두 자리까지의 문자를 선택하고, 이를 "월" 열에 저장합니다. 예를 들어, "2005.01"에서 "01"을 추출합니다.


In [ ]:
df_comp["연도"] = df_comp["시점"].astype(str).str[:4].astype(int)
df_comp["월"] = df_comp["시점"].map(lambda x: x.split(".")[1][:2]).astype(int)
df_comp

In [ ]:
# 데이터의 개수 => 통계치가 아님에 주의
# 데이터 전처리가 잘 되었는지 확인하는 용도
pd.crosstab(df_comp["연도"], df_comp["월"])

In [ ]:
year_comp = pd.crosstab(index=df_comp["연도"],
            columns=df_comp["항공사"],
            values=df_comp["여객_명"],
            aggfunc="sum").fillna(0)
year_comp.style.background_gradient(axis=None).format("{:,.0f}")

In [ ]:
df_comp["항공사"].unique()

In [ ]:
year_comp[['제주항공', '진에어', '에어부산', '이스타항공', '티웨이항공',
       '에어인천', '에어서울', '플라이강원', '에어로케이항공']].plot(figsize=(12, 3), title="저가항공 연도별 여객 수")

In [ ]:
year_comp[['제주항공', '진에어', '에어부산', '이스타항공', '티웨이항공',
       '에어인천', '에어서울', '플라이강원', '에어로케이항공']].plot(figsize=(12, 9), title="저가항공 연도별 여객 수", subplots=True);

https://matplotlib.org/stable/tutorials/colors/colormaps.html

In [ ]:
print(plt.colormaps())

In [ ]:
year_comp = pd.crosstab(index=df_comp["연도"],
            columns=df_comp["항공사"],
            values=df_comp["운항_편"],
            aggfunc="sum").fillna(0)
year_comp.style.background_gradient(axis=None, cmap="Greens").format("{:,.0f}")

In [ ]:
year_comp = pd.crosstab(index=df_comp["연도"],
            columns=df_comp["항공사"],
            values=df_comp["화물_톤"],
            aggfunc="sum").fillna(0)
year_comp.style.background_gradient(axis=None, cmap="Oranges").format("{:,.0f}")

## 한국공항공사, 인천국제공항공사,「항공통계」, 국제선 지역별 통계
* 데이터 : https://kosis.kr/statHtml/statHtml.do?orgId=381&tblId=DT_920005_B005&conn_path=I2
* 통계 설명 자료 : https://www.narastat.kr/metasvc/index.do?orgId=381&confmNo=920005&kosisYn=Y

In [ ]:
df_region = pd.read_csv("kosis-airline-region.csv", encoding="cp949")
df_region.shape

In [ ]:
df_region.columns = ['지역', '시점', '도착출발', '운항_편', '여객_명', '화물_톤']
df_region.head(2)

In [ ]:
df_region.info()

In [ ]:
df_region.isnull().sum()

이용자용 통계정보보고서_항공통계2018.pdf

In [ ]:
# 1. 지역별 운항 편 수 분석
region_flight_counts = df_region.groupby("지역")["운항_편"].sum().sort_values(ascending=False)
region_flight_counts.plot(kind="bar", figsize=(12, 3), rot=0)
plt.title("지역별 운항 편 수")
plt.xlabel("지역")
plt.ylabel("운항 편 수")
plt.show()

In [ ]:
# ESC + m + y => 코드셀로 변환
# 2. 월별 운항 편 수 추이 시각화
df_region["시점"] = pd.to_datetime(df_region["시점"].astype(str), format="%Y.%m")
df_region["시점"]

In [ ]:
df_region["연도"] = df_region["시점"].dt.year
df_region["월"] = df_region["시점"].dt.month
df_region

In [ ]:
monthly_flight_counts = df_region.groupby(df_region["시점"].dt.strftime("%Y-%m"))["운항_편"].sum()
monthly_flight_counts.plot(kind="line", marker="o")
plt.title("월별 운항 편 수 추이")
plt.xlabel("월")
plt.ylabel("운항 편 수")
plt.xticks(rotation=45)
plt.show()

In [ ]:
df_region[df_region["월"] == 1]

In [ ]:
# 3. 도착 vs. 출발 운항 비교
arrival_vs_departure = df_region.groupby("도착출발")["운항_편"].sum()
arrival_vs_departure.plot(kind="pie", autopct='%1.1f%%')
plt.title("도착 vs. 출발 운항 편 수 비교")
plt.ylabel("")
plt.show()

In [ ]:
df_region

In [ ]:
pd.crosstab(index=df_region["도착출발"],
            columns=df_region["연도"],
            values=df_region["여객_명"], aggfunc="sum"
            ).T.plot(kind="bar", figsize=(12, 3), title="연도별 도착 출발 여객 명");

In [ ]:
# 변경된 코드: groupby를 사용한 운항 편 수 집계
df_region.groupby(["도착출발", df_region["시점"].dt.year])["운항_편"].sum().unstack()

In [ ]:

# "지역" 열을 기준으로 데이터 그룹화
grouped_by_region = df_region.groupby("지역")

# 각 지역별 운항 편수, 여객 수, 화물 운송량의 합계 계산
region_stats = grouped_by_region[["운항_편", "여객_명", "화물_톤"]].sum()

# 각 지역별 운항 편수, 여객 수, 화물 운송량의 합계를 막대 그래프로 시각화
region_stats.plot(kind="bar", figsize=(12, 6))
plt.title("지역별 운항 편수, 여객 수, 화물 운송량")
plt.xlabel("지역")
plt.ylabel("합계")
plt.xticks(rotation=45)
plt.legend(["운항 편수", "여객 수", "화물 운송량"])
plt.show()

In [ ]:
df_region.tail()

In [ ]:
year_region = pd.crosstab(index=df_region["연도"], columns=df_region["지역"],
            values=df_region["운항_편"], aggfunc="sum").fillna(0)
year_region.style.background_gradient(cmap="Oranges", axis=None).format("{:,.0f}")

In [ ]:
year_region.iloc[-5:][["일본", "중국", "아시아", "미주"]].plot(figsize=(12, 3))

In [ ]:
year_region.iloc[-5:][["일본", "중국", "아시아", "미주"]].plot(kind="bar")

In [ ]:
year_region.iloc[-1][["일본", "중국", "아시아", "미주"]].sort_values().plot(figsize=(12, 3), kind="bar", rot=0)

In [ ]:
year_region.loc[[2019, 2020, 2023]].T.plot(kind="bar")

In [ ]:
df_month = df_region[df_region["월"] == 3]
year_region_m = pd.crosstab(index=df_month["연도"], columns=df_month["지역"],
            values=df_month["운항_편"], aggfunc="sum").fillna(0)
year_region_m.style.background_gradient(cmap="Oranges", axis=None).format("{:,.0f}")

In [ ]:
df_month = df_region[df_region["월"] == 3]
year_region_m = pd.crosstab(index=df_month["연도"], columns=df_month["지역"],
            values=df_month["화물_톤"], aggfunc="sum").fillna(0)
year_region_m.style.background_gradient(cmap="Oranges", axis=None).format("{:,.0f}")

In [ ]:
ym = pd.crosstab(df_region["연도"], df_region["월"], values=df_region["화물_톤"], aggfunc="sum")
ym.style.background_gradient(cmap="Oranges", axis=None).format("{:,.0f}")